In [3]:
import numpy as np
import matplotlib.pyplot as plt
from termcolor import colored

import tensorflow as tf
import tensorflow_datasets as tfds

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Flatten, Dense, Activation

from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import SGD, Adam

from tensorflow.keras.metrics import Mean, SparseCategoricalAccuracy

## 1. Dataset 로드 & 전처리 함수 정의



In [6]:
def get_mnist_ds():
    (train_validation_ds, test_ds), ds_info = tfds.load(name='mnist',
                                                        shuffle_files=True,
                                                        as_supervised=True,
                                                        split=['train', 'test'],
                                                        with_info=True)
    n_train_validation = ds_info.splits['train'].num_examples
    
    train_ratio = 0.8
    n_train = int(n_train_validation*train_ratio)
    n_validation = n_train_validation - n_train

    train_ds = train_validation_ds.take(n_train)
    remaining_ds = train_validation_ds.skip(n_train)
    validation_ds = remaining_ds.take(n_validation)

    return train_ds, validation_ds, test_ds

In [7]:
def standardization(TRAIN_BATCH_SIZE, TEST_BATCH_SIZE):
    global train_ds, validation_ds, test_ds

    def stnd(images, labels):
        images = tf.cast(images, tf.float32) / 255.
        return [images, labels]
    
    train_ds = train_ds.map(stnd).shuffle(1000).batch(TRAIN_BATCH_SIZE)
    validation_ds = validation_ds.map(stnd).batch(TEST_BATCH_SIZE)
    test_ds = test_ds.map(stnd).batch(TEST_BATCH_SIZE)

## 2. 모델 정의

In [10]:
class MNIST_Classifier(Model):
    def __init__(self):
        super(MNIST_Classifier, self).__init__()
        
        self.flatten = Flatten()
        self.d1 = Dense(64, activation='relu')
        self.d2 = Dense(10, activation='softmax')

    def call(self, x):
        x = self.flatten(x)
        x = self.d1(x)
        x = self.d2(x)
        return x

In [11]:
def load_metrics():
    global train_loss, train_acc
    global validation_loss, validation_acc
    global test_loss, test_acc

    train_loss = Mean()
    validation_loss = Mean()
    test_loss = Mean()

    train_acc = SparseCategoricalAccuracy()
    validation_acc = SparseCategoricalAccuracy()
    test_acc = SparseCategoricalAccuracy()

## 3. 학습/검증/테스트 및 로깅 함수 정의

In [48]:
@tf.function
def trainer():
    global train_ds, model, loss_object, optimizer
    global train_loss, train_acc

    for images, labels in train_ds:
        with tf.GradientTape() as tape:
            predictions = model(images)
            loss = loss_object(labels, predictions)

        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))

        train_loss(loss)
        train_acc(labels, predictions)

In [49]:
@tf.function
def validation():
    global validation_ds, model, loss_object
    global validation_loss, validation_acc

    for images, labels in validation_ds:
        predictions = model(images)
        loss = loss_object(labels, predictions)

        validation_loss(loss)
        validation_acc(labels, predictions)

In [47]:
# @tf.function   # print()문을 사용할 경우, tf.function 데코레이터는 사용하지 못한다
def tester():
    global test_ds, model, loss_object
    global test_loss, test_acc

    for images, labels in test_ds:
        predictions = model(images)
        loss = loss_object(labels, predictions)

        test_loss(loss)
        test_acc(labels, predictions)

    print('=====TEST=====')
    template = 'Test Loss: {:.4f}\t Test Accuracy: {:.2f}%\n'
    print(template.format(test_loss.result(), test_acc.result()*100))

In [35]:
def train_reporter():
    global epoch
    global train_loss, train_acc
    global validation_loss, validation_acc

    print(colored('Epoch', 'red', 'on_white'), epoch+1)
    template = 'Train Loss: {:.4f}\t Train Accuracy: {:.2f}%\n'+\
        'Validation Loss: {:.4f}\t Validation Accuracy: {:.2f}%\n'
    print(template.format(train_loss.result(),
                          train_acc.result()*100,
                          validation_loss.result(),
                          validation_acc.result()*100))
    train_acc.reset_states()
    train_loss.reset_states()
    validation_loss.reset_states()
    validation_acc.reset_states()

## 3. 학습

In [16]:
EPOCHS = 10
LR = 0.001
TRAIN_BATCH_SIZE = 16
TEST_BATCH_SIZE = 32

In [17]:
train_ds, validation_ds, test_ds = get_mnist_ds()
standardization(TRAIN_BATCH_SIZE, TEST_BATCH_SIZE)

In [50]:
model = MNIST_Classifier()
loss_object = SparseCategoricalCrossentropy()
optimizer = SGD(learning_rate=LR)

load_metrics()

In [51]:
for epoch in range(EPOCHS):
    trainer()
    validation()
    train_reporter()

tester()

Epoch 1
Train Loss: 1.5090	 Train Accuracy: 60.45%
Validation Loss: 0.9664	 Validation Accuracy: 77.88%

Epoch 2
Train Loss: 0.7610	 Train Accuracy: 82.30%
Validation Loss: 0.6351	 Validation Accuracy: 84.47%

Epoch 3
Train Loss: 0.5638	 Train Accuracy: 85.85%
Validation Loss: 0.5185	 Validation Accuracy: 86.69%

Epoch 4
Train Loss: 0.4805	 Train Accuracy: 87.40%
Validation Loss: 0.4596	 Validation Accuracy: 87.88%

Epoch 5
Train Loss: 0.4336	 Train Accuracy: 88.39%
Validation Loss: 0.4233	 Validation Accuracy: 88.62%

Epoch 6
Train Loss: 0.4030	 Train Accuracy: 89.06%
Validation Loss: 0.3983	 Validation Accuracy: 89.14%

Epoch 7
Train Loss: 0.3809	 Train Accuracy: 89.52%
Validation Loss: 0.3802	 Validation Accuracy: 89.55%

Epoch 8
Train Loss: 0.3641	 Train Accuracy: 89.93%
Validation Loss: 0.3657	 Validation Accuracy: 89.99%

Epoch 9
Train Loss: 0.3505	 Train Accuracy: 90.27%
Validation Loss: 0.3540	 Validation Accuracy: 90.20%

Epoch 10
Train Loss: 0.3391	 Train Accuracy: 90.55%
Val